In [1]:
from PIL import Image
from torchvision import transforms

import torch
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
import random

## Preprocessing the ImageNet Examples

In [2]:
from os import listdir
from os.path import isfile, join

prefix =  "./greek_examples/"

onlyfiles = [prefix+f for f in listdir("./greek_examples") if isfile(join("./greek_examples", f))]

In [3]:
print(onlyfiles)

['./greek_examples/images (98).jpg', './greek_examples/images - 2020-05-31T060902.924.jpg', './greek_examples/images - 2020-05-31T060907.852.jpg', './greek_examples/images - 2020-05-31T060913.371.jpg', './greek_examples/images - 2020-05-31T060910.906.jpg', './greek_examples/images (75).jpg', './greek_examples/images - 2020-05-31T060921.702.jpg', './greek_examples/images - 2020-05-31T060919.807.jpg', './greek_examples/images - 2020-05-31T060912.961.jpg', './greek_examples/images - 2020-05-31T060909.659.jpg', './greek_examples/images - 2020-05-31T060910.486.jpg', './greek_examples/images (13).jpg', './greek_examples/2Q__ (5).jpg', './greek_examples/images - 2020-05-31T060903.327.jpg', './greek_examples/images (58).jpg', './greek_examples/images - 2020-05-31T060927.237.jpg', './greek_examples/images (31).jpg', './greek_examples/images - 2020-05-31T060925.776.jpg', './greek_examples/2Q__ (3).jpg', './greek_examples/images - 2020-05-31T060914.390.jpg', './greek_examples/images - 2020-05-31T

In [4]:
from PIL import Image
from torchvision import transforms
import numpy as np

import torch
import matplotlib.pyplot as plt

In [5]:
#im1 = im1.save("geeks.jpg") 
img_list = []

In [6]:
for i, file_path in enumerate(onlyfiles):
    img_list.append(Image.open(file_path))

In [7]:
def trans_pil2ImageNet(img):
    trans_resize = transforms.Resize(size=(256, 256))
    resized_img = trans_resize(img)
    return resized_img

def trans_randRotate(img):
    trans_rotate = transforms.RandomAffine(degrees=(-180, 180))
    rotated_img = trans_rotate(img)
    return rotated_img

def trans_pil2tensor(pil_img):
    trans_pil2t = transforms.ToTensor()
    transformed_numpy = trans_pil2t(pil_img).numpy()  # convert to HWC
    return (transformed_numpy * 255).astype(np.uint8)

def trans_rand_crop_flip(pil_img):
    transformer_rand_crop = transforms.RandomCrop(32, padding=4)
    transformer_rand_flip = transforms.RandomHorizontalFlip()
    croped = transformer_rand_crop(pil_img)
    croped_fliped = transformer_rand_flip(croped)
    return croped_fliped

In [8]:
print("Num of collected Greek images: {}".format(len(img_list)))

Num of collected Greek images: 167


## Augment the collected Greek Images

In [9]:
_SEED = 0
# seed = 1234
random.seed(_SEED)
torch.manual_seed(_SEED)
#torch.cuda.manual_seed(seed)
#torch.cuda.manual_seed_all(seed)
np.random.seed(_SEED)

In [10]:
indices_greek = np.arange(len(img_list))
permed_indices_greek = np.random.permutation(indices_greek)

In [11]:
train_adv_indices = permed_indices_greek[0:67]
train_honest_indices = permed_indices_greek[67:134]
test_indices = permed_indices_greek[134:]

print("Train adv indices: {}, Num Train assigned: {}".format(train_adv_indices, len(train_adv_indices)))
print()
print("Train honest indices: {}, Num Train assigned: {}".format(train_honest_indices, len(train_honest_indices)))
print()
print("Test indices: {}, Num Test indices: {}".format(test_indices, len(test_indices)))

Train adv indices: [150  56 123  60 100  51   7   5  86 135  54  94 156 101  73  45  74  97
 139  96 107 106 149 165  55 151  26 142 154  33   8 108 122  37  61  44
  24  63  89  19 130 148 138  16  66 136  40 158  22  71  95 121 104 109
  27  90 129  93 112 113  18 119 164  30  83  62   2], Num Train assigned: 67

Train honest indices: [ 59 111  92  43  10  80 161 126  98 153 134 160  50 110 152  64 116 118
  69  49  48  85  13 132  23 124  20  15  78  52  76   3 166 146   6  68
  75  84 125  12 131 143  14   0  91 144  46  11 102  35  57  41 163  65
   1 120 137 155  42 105   4 145  17  38  53 133 128], Num Train assigned: 67

Test indices: [ 34  28 114  31 141 127 147  32  29  99  82  79 115 162  72  77  25  81
 157 159  39  58 140  88  70  87  36  21   9 103  67 117  47], Num Test indices: 33


In [12]:
transformed_img_list_train_adv = []
transformed_img_list_train_honest = []

In [13]:
fraction=3
cand_angles = [90 * i for i in range(1, fraction+1)]

for train_adv_idx in train_adv_indices:
    img_train_adv = img_list[train_adv_idx]
    img_imagenet_size_train_adv = trans_pil2ImageNet(img_train_adv)
    transformed_img_list_train_adv.append(img_imagenet_size_train_adv)
    for cad_ang in cand_angles:
        img_rotated_train_adv = transforms.functional.rotate(img_imagenet_size_train_adv, cad_ang)
        transformed_img_list_train_adv.append(img_rotated_train_adv)

In [14]:
for trans_idx_train_adv, trans_imgs_train_adv in enumerate(transformed_img_list_train_adv):
    trans_imgs_train_adv.save("./greek_preprocessed/train/processed_train_img{}.JPEG".format(trans_idx_train_adv))

In [15]:
fraction=3
cand_angles = [90 * i for i in range(1, fraction+1)]

for train_honst_idx in train_honest_indices:
    img_train_honst = img_list[train_honst_idx]
    img_imagenet_size_train_honst = trans_pil2ImageNet(img_train_honst)
    transformed_img_list_train_honest.append(img_imagenet_size_train_honst)
    for cad_ang in cand_angles:
        img_rotated_train_honst = transforms.functional.rotate(img_imagenet_size_train_honst, cad_ang)
        transformed_img_list_train_honest.append(img_rotated_train_honst)

In [16]:
for trans_idx_train_honst, trans_imgs_train_honst in enumerate(transformed_img_list_train_honest):
    trans_imgs_train_honst.save("./greek_preprocessed/train_honest/processed_train_img{}.JPEG".format(trans_idx_train_honst))

In [17]:
transformed_img_list_test = []

In [18]:
fraction=3
cand_angles = [90 * i for i in range(1, fraction+1)]

for test_idx in test_indices:
    img_test = img_list[test_idx]
    img_imagenet_size_test = trans_pil2ImageNet(img_test)
    transformed_img_list_test.append(img_imagenet_size_test)
    for cad_ang in cand_angles:
        img_rotated_test = transforms.functional.rotate(img_imagenet_size_test, cad_ang)
        transformed_img_list_test.append(img_rotated_test)

In [19]:
for trans_idx_test, trans_imgs_test in enumerate(transformed_img_list_test):
    trans_imgs_test.save("./greek_preprocessed/test/processed_test_img{}.JPEG".format(trans_idx_test))